In [98]:
from prep_data import Data_prepare
import numpy as np 
import os 
# import prep_data

###  Data preprocessing 
1. covert mat data to numpy array 
2. combine male and female data to one array 
3. if needed, remove certain rows 
4. get postive and negative of Xs 
    - order difference or net vs neg 
5. reshape each X: 128*8*2 = 2048 
    - input: data 
6. generate labels for x 
    - input: data and label_type (0:zeros, 1: ones )
7. combine net and neg 
    - input: netX,netY,negX,negY

In [99]:
dp = Data_prepare() 
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
# netural vs postive 
# pos = [[0,0],[0,1],[1,2],[1,3],[2,0],[2,1],[2,2],[2,3]]
# neg = [[0,2],[0,3],[1,0],[1,1],[3,0],[3,1],[3,2],[3,3]] 
m_file = "./raw_data/Emotrans1_Boy_data_preprocessed_42.mat" 
f_file = "./raw_data/Emotrans1_girl_data_preprocessed_42.mat"
index = [11,18,36]
df = dp.combine_male_female(m_file,f_file)
df = dp.remove_person(df,index)
pos_df = dp.get_pos_or_neg(df,pos)
neg_df = dp.get_pos_or_neg(df,neg)
pos_sequeezed = dp.squeeze_feature_size(pos_df)
neg_sequeezed = dp.squeeze_feature_size(neg_df)
pos_labels = dp.generate_labels(pos_sequeezed,1)
neg_labels = dp.generate_labels(neg_sequeezed,0)
x,y = dp.combine_net_neg(pos_sequeezed,pos_labels,neg_sequeezed,neg_labels)

In [100]:
## (40,) -> 40*36*2048
def flatten(data):
    res = data[0]
    for i in range(1,data.shape[0]):
        res = np.append(res,data[i],axis=0)
    return res 
new_x = flatten(x)
new_y = flatten(y)

In [101]:
print(new_x.shape,new_y.shape)


(1374, 2048) (1374, 1)


#### Normalize x with standardScaler

In [102]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(new_x)
new_x = scaler.transform(new_x)

In [103]:
print(new_x.shape,new_y.shape)

(1374, 2048) (1374, 1)


In [134]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(new_x, new_y, test_size = 0.1, random_state = 4)

In [135]:
print(x_train.shape,y_train.shape)

(1236, 2048) (1236, 1)


In [136]:
## reshape x_train and x_test to 3D for LSTM 
x_train = np.reshape(x_train, (x_train.shape[0],1,new_x.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0],1,new_x.shape[1]))

In [137]:
# y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
# y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [138]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(1236, 1, 2048) (138, 1, 2048)
(1236, 1) (138, 1)


#### LSTM model 

In [139]:
import tensorflow as tf
from tensorflow.keras import Sequential

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
tf.keras.backend.clear_session()

model = Sequential()
model.add(LSTM(512, input_shape=(1,2048),activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256,activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128,activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128,activation="relu",return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(16,activation="relu"))
model.add(Dropout(0.2))
# model.add(LSTM(50))
# model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
from keras.optimizers import SGD
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 512)            5244928   
_________________________________________________________________
dropout (Dropout)            (None, 1, 512)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 256)            787456    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 256)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 128)            197120    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 128)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 128)            1

In [140]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 50, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/50
39/39 [==============================] - 2s 58ms/step - loss: 0.7017 - accuracy: 0.5858 - val_loss: 0.5693 - val_accuracy: 0.7319
Epoch 2/50
39/39 [==============================] - 1s 34ms/step - loss: 0.6533 - accuracy: 0.6788 - val_loss: 0.5839 - val_accuracy: 0.7391
Epoch 3/50
39/39 [==============================] - 1s 34ms/step - loss: 0.6793 - accuracy: 0.7087 - val_loss: 0.4724 - val_accuracy: 0.7536
Epoch 4/50
39/39 [==============================] - 1s 35ms/step - loss: 0.5993 - accuracy: 0.7193 - val_loss: 0.4537 - val_accuracy: 0.7754
Epoch 5/50
39/39 [==============================] - 1s 35ms/step - loss: 0.6023 - accuracy: 0.7751 - val_loss: 0.4327 - val_accuracy: 0.7899
Epoch 6/50
39/39 [==============================] - 1s 34ms/step - loss: 0.5284 - accuracy: 0.7896 - val_loss: 0.5186 - val_accuracy: 0.7899
Epoch 7/50
39/39 [==============================] - 1s 34ms/step - loss: 0.4642 - accuracy: 0.8091 - val_loss: 0.5392 - val_accuracy: 0.7464
Epoch 8/50
39

In [ ]:
### leave one out 


In [60]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [62]:
# %tensorboard --logdir logs

#### CNN 

In [141]:
## reshape 
x_train = np.reshape(x_train, (x_train.shape[0],new_x.shape[1],1))
x_test = np.reshape(x_test, (x_test.shape[0],new_x.shape[1],1))

In [142]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(1236, 2048, 1) (138, 2048, 1)
(1236, 1) (138, 1)


In [143]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.recurrent import LSTM
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers.wrappers import TimeDistributed
model = models.Sequential()
model.add(layers.Conv1D(16, 10, activation='relu',input_shape=(2048,1)))
model.add(layers.Conv1D(16, 3, activation='relu'))
model.add(layers.MaxPool1D(2))
model.add(layers.Flatten())

model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(3,activation="softmax"))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())
# input_shape = (trial_length, nchan, 1)
# l1 = 0
# model = Sequential()
# model.add(Conv2D(40, (30, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=(2048,1)))
# model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
# model.add(AveragePooling2D((30, 1), strides=(15, 1)))
# model.add(Flatten())
# model.add(Dense(80, activation="relu"))
# model.add(Dense(2, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2039, 16)          176       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2037, 16)          784       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1018, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 16288)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1628900   
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 1,630,163
Trainable params: 1,630,163
Non-trainable params: 0
____________________________________________

In [145]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 30, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/30
39/39 [==============================] - 1s 23ms/step - loss: 0.2049 - accuracy: 0.9231 - val_loss: 0.6175 - val_accuracy: 0.7319
Epoch 2/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1816 - accuracy: 0.9304 - val_loss: 0.6375 - val_accuracy: 0.7319
Epoch 3/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1442 - accuracy: 0.9628 - val_loss: 0.7088 - val_accuracy: 0.7536
Epoch 4/30
39/39 [==============================] - 1s 23ms/step - loss: 0.1179 - accuracy: 0.9636 - val_loss: 0.7885 - val_accuracy: 0.7464
Epoch 5/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1055 - accuracy: 0.9741 - val_loss: 0.9229 - val_accuracy: 0.7536
Epoch 6/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1149 - accuracy: 0.9668 - val_loss: 0.8137 - val_accuracy: 0.7681
Epoch 7/30
39/39 [==============================] - 1s 22ms/step - loss: 0.1165 - accuracy: 0.9604 - val_loss: 0.8468 - val_accuracy: 0.7319
Epoch 8/30
39

#### CRNN

In [96]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.recurrent import LSTM
from keras import regularizers
from keras.callbacks import ModelCheckpoint
from keras.layers.wrappers import TimeDistributed

model = models.Sequential()
model.add(layers.Conv1D(16, 10, activation='relu',input_shape=(2048,1)))
model.add(layers.Conv1D(16, 3, activation='relu'))
model.add(layers.MaxPool1D(2))
# model.add(AveragePooling2D((5, 1), strides=(5, 1)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(40, activation="sigmoid", dropout=0.25, return_sequences=False))
model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(2,activation="softmax"))
## rmsprop
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())
# input_shape = (trial_length, nchan, 1)
# l1 = 0
# model = Sequential()
# model.add(Conv2D(40, (30, 1), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="same", input_shape=(2048,1)))
# model.add(Conv2D(40, (1, nchan), activation="relu", kernel_regularizer=regularizers.l1(l1), padding="valid"))
# model.add(AveragePooling2D((30, 1), strides=(15, 1)))
# model.add(Flatten())
# model.add(Dense(80, activation="relu"))
# model.add(Dense(2, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 2039, 16)          176       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 2037, 16)          784       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1018, 16)          0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 1018, 16)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 40)                9120      
_________________________________________________________________
dense_9 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_10 (Dense)             (None, 2)               

In [97]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(x_train, y_train, epochs = 20, validation_data= (x_test, y_test),callbacks=[tensorboard_callback])
score, acc = model.evaluate(x_test, y_test)

Epoch 1/20
40/40 [==============================] - 22s 562ms/step - loss: 0.6987 - accuracy: 0.4545 - val_loss: 0.6895 - val_accuracy: 0.5461
Epoch 2/20
40/40 [==============================] - 20s 493ms/step - loss: 0.6962 - accuracy: 0.5028 - val_loss: 0.6881 - val_accuracy: 0.5461
Epoch 3/20
40/40 [==============================] - 18s 438ms/step - loss: 0.6938 - accuracy: 0.5028 - val_loss: 0.6887 - val_accuracy: 0.5745
Epoch 4/20
40/40 [==============================] - 18s 442ms/step - loss: 0.6892 - accuracy: 0.5368 - val_loss: 0.6769 - val_accuracy: 0.5532
Epoch 5/20
40/40 [==============================] - 17s 435ms/step - loss: 0.6792 - accuracy: 0.5558 - val_loss: 0.6721 - val_accuracy: 0.5248
Epoch 6/20
40/40 [==============================] - 17s 436ms/step - loss: 0.6562 - accuracy: 0.6144 - val_loss: 0.6300 - val_accuracy: 0.6738
Epoch 7/20
40/40 [==============================] - 17s 435ms/step - loss: 0.6445 - accuracy: 0.6207 - val_loss: 0.6263 - val_accuracy: 0.6383